In [1]:
# Dependencies
from bs4 import BeautifulSoup as bs
from splinter import Browser
import pandas as pd
import requests

In [2]:
# Scrape the Nasa Mars News Site and collect the latest news title and paragraph text
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

url = 'https://mars.nasa.gov/news/'
browser.visit(url)

# HTML Object
html = browser.html

# Parse HTML with Beautiful Soup
soup = bs(html, 'html.parser')


# Retrieve the latest element that contains news title and news paragraph
news_title = soup.find('div', class_='content_title').find('a').text
news_p = soup.find('div', class_='article_teaser_body').text

# Display scrapped data 
print(news_title)
print(news_p)

NASA's Curiosity Mars Rover Finds a Clay Cache
The rover recently drilled two samples, and both showed the highest levels of clay ever found during the mission.


In [3]:
# Visit the url for JPL Featured Space Image
image_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(image_url)

# HTML Object 
image_html = browser.html

# Parse HTML with Beautiful Soup
soup = bs(image_html, 'html.parser')

# Scrape Path for the featured image starting with the partial path of the url
partial_address = soup.find_all('a', class_='fancybox')[0].get('data-fancybox-href').strip()

# Combine the root url to get the full address
featured_image_url = "https://www.jpl.nasa.gov"+partial_address

# Print to check the full URL
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA18905_ip.jpg


In [4]:
# Visit Mars Weather twitter for weather data
weather_url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(weather_url)

# HTML Object 
weather_html = browser.html

# Parse HTML with Beautiful Soup
soup = bs(weather_html, 'html.parser')

# First, find a tweet with the data-name Mars Weather
mars_weather_tweet = soup.find('div', attrs={"class": "tweet", "data-name": "Mars Weather"})

# Next, search within the tweet for the p tag containing the tweet text
mars_weather = mars_weather_tweet.find('p', 'tweet-text').get_text()
mars_weather

'InSight sol 182 (2019-06-01) low -99.7ºC (-147.5ºF) high -22.6ºC (-8.7ºF)\nwinds from the SSE at 5.7 m/s (12.7 mph) gusting to 15.9 m/s (35.5 mph)\npressure at 7.50 hPapic.twitter.com/NmzIAqOiDG'

In [5]:
# Visit the Mars Facts webpage
facts_url = 'http://space-facts.com/mars/'

# Use Panda's `read_html` to parse the url
mars_facts = pd.read_html(facts_url)

# Find the mars facts DataFrame in the list of DataFrames as assign it to `mars_df`
mars_df = mars_facts[0]

# Assign the columns `['Description', 'Value']`
mars_df.columns = ['Description','Value']

# Set the index to the `Description` column
mars_df.set_index('Description', inplace=True)

# Save html code to folder Assets
mars_df.to_html()

data = mars_df.to_dict(orient='records')  # Here's our added param..

# Display mars_df
mars_df

,Value
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers
